In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import nibabel as nib
import glob
import os
from nilearn.plotting import plot_stat_map, view_img, plot_glass_brain
from nilearn.image import concat_imgs, smooth_img, mean_img, math_img, resample_to_img
from nilearn.glm.second_level import SecondLevelModel
from nilearn.glm import threshold_stats_img
from nilearn.glm.second_level import non_parametric_inference

In [ ]:
reg = 'valBundleSum_par'
l2_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/bids_nifti_wface/derivatives/nilearn/glm/level2/model3_reg-rt0'
l2_imgs = glob.glob(os.path.join(l2_path, '*/*/*%s*'%(reg)))

## How can l2 z-maps look very constrained but l3 randomise with massive swaths of activation?

- How l3 t values be larger than the largest t values in the individual l2 mean maps?
    - L2 mean maps are generated by `nilearn.glm.second_level.SecondLevelModel.compute_contrast` with argument `output_type="z_score"`
    - L3 t values are computed using FSL's `randomise`  

Previous hypotheses:

- Is it because `rabndomise` uses enhanced TFCE statistics instaed of raw T stats?  
    - No. When I tried voxelwise setting `tfce=False` and checked the min and max values in `randomise_results.outputs.tstat_files` they were the same as the outputs using tfce.  

- Is it because of smoothing?
    - No. Smoothing l2 images made values only smaller.

Current hypothesis:

- Statistic mismatch between levels of analysis.
    - FSL's `randomise` expects `cope` images. These should be beta weights. What `output_type` do these correspond to in `nilearn` terminology?
    - What is the correct input type for l2 and l3 analyses in `nilearn`?
    - What does a max T distribution look like in `nilearn`s permutation testing framework?

## `compute_contrast` output types

- Run level 1 analysis on single subject single run
- Run `compute_contrast` for each `output_type`
- Pick a single voxel
- Run GLM on it and compare the value for this voxel in each contrast map to the GLM summary

## Run level3 analysis using nilearn instead of randomise

In [ ]:
# 1's for all runs to get average for subject per contrast
design_matrix = pd.DataFrame([1] * len(l2_imgs), columns=['intercept'])
model = SecondLevelModel(smoothing_fwhm=5.0)

model = model.fit(l2_imgs, design_matrix=design_matrix)

# z_map = model.compute_contrast(output_type='z_score')
z_map = model.compute_contrast(output_type='stat')


Raw map thresholded at an arbitrary large value. What nilearn reports for `output_type = "stat"` is comparable to randomise outputs.

What statistic is reported when `output_type = "stat"`? 

Could this be why things look weird when using randomise? I'm feeding in `z_score` maps insteaf og `stat` maps? What input does randomise expect? `cope` images, contrast of parameters (beta weights)?




In [ ]:
plot_stat_map(z_map, threshold=3, draw_cross=False)


In [ ]:
neg_log_pvals_permuted_ols_unmasked = non_parametric_inference(l2_imgs,
                             design_matrix=design_matrix,
                             model_intercept=True, n_perm=5000,
                             two_sided_test=False,
                             smoothing_fwhm=5.0, n_jobs=1)

In [ ]:
plot_stat_map(
    neg_log_pvals_permuted_ols_unmasked, colorbar=True, 
    threshold=1, draw_cross=False, cut_coords = (-6, -95, 14))
# The neg-log p-values obtained with non parametric testing are capped at 3
# when the number of permutations is 1e3.

What is the max T distribution that lead to the map above? Should be able to get it with the `nilearn.mass_univariate.permuted_ols` function.

### Compared to the randomise output

In [ ]:
plot_stat_map(os.path.join(l3_path, 'rand_tfce_tstat1_pos_overall-mean_reward_par_model3_reg-rt0.nii.gz'), threshold = 4.6, draw_cross=False)